# ML task 1

In [1]:
# import hypertools as hyp
import pandas as pd

RS = 42

%matplotlib inline

Загрузим и посмотрим на датасет:

In [2]:
df = pd.read_csv("dataset.csv")
print(df.isnull().values.any())
print(df["target"].value_counts())
df.describe()

False
target
0    2661
1     539
Name: count, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,796,797,798,799,800,801,802,803,804,target
count,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,...,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000,3200.000000
mean,4.905736,1.004077,4.905259,-16.764958,11.378823,0.985953,0.991219,4.843143,4.818068,1.019979,...,4.971858,1.021051,15.134639,1.001243,66.538199,4.898069,4.927336,25.268862,97.299306,0.168437
std,572.099724,1.006358,572.239569,1144.194017,1166.478566,0.998380,1.021583,586.901682,572.144211,1.021059,...,572.067999,1.006967,0.720456,1.011120,0.722748,572.089702,572.102289,1138.873192,0.721738,0.374313
min,-1985.373872,-0.365879,-1979.222507,-1997.880619,-1999.071536,-0.371556,-0.326992,-1985.977156,-1978.848358,-0.359753,...,-1979.513449,-0.342419,13.738149,-0.267957,65.139060,-1979.963658,-1985.035268,-1998.404272,95.956766,0.000000
25%,1.733205,0.048172,1.689857,-1005.597646,-1005.410989,0.041556,0.044230,1.769651,1.632500,0.053863,...,1.736377,0.052952,14.417427,0.043592,65.809713,1.692510,1.689074,-936.466096,96.601876,0.000000
50%,5.066410,0.966576,5.005792,-33.068556,12.529704,0.961443,0.960883,5.196335,5.050662,0.969711,...,5.113488,0.968487,15.183182,0.968037,66.598725,5.003343,4.963494,22.042498,97.355167,0.000000
75%,8.302775,1.853334,8.283948,986.587994,1039.632650,1.848203,1.857232,8.377849,8.176830,1.865839,...,8.302549,1.882305,15.840340,1.833787,67.234482,8.332029,8.158775,1018.741960,98.005873,0.000000
max,1999.164279,6.018990,1998.083977,1999.948257,1998.248312,5.108544,6.127216,2006.079911,1996.010797,7.010780,...,1997.981103,6.964271,16.347344,5.969353,67.753568,1993.521727,2002.557362,1996.266269,98.530184,1.000000


Выводы:
1) Датасет не сбалансирован, больше данных с target=0, надо попробовать {over|under}sampling, метрику брать не просто accuracy, а что-нибудь типа auc
2) Много фичей, надо будет уменьшить количество
3) Тут была визуализация с помощью hypertools, по техническим причинам не сохранилась. На ней увидел кластера объектов, думаю, KNN отработает
4) Пробовать нужно много классификаторов с разными параметрами, можно запускать GridSearch

Собрал, запустил скрипт [prepare_dataset.py](./prepare_dataset.py) для подбора и сохранения параметров. Получил [learn1.log](./learn1.log)*, сделал выводы:

1. LogisticRegression - убрать, слишком простой
2. добавить редкуцию до 3 компонентов где можно
3. KNN: добавить 2 и 3 соседа
4. Убрать RadiusNeighborsClassifier
5. LLE: понизить количество соседей (до 10)
6. Isomap можно не тестить
7. (ADASYN(), KernelPCA(kernel='poly')) лучше, чем (ADASYN(), KernelPCA(kernel='rbf'))
8. Без самплинга не запускаю
9. Уменьшить кол-во компонентов для KernelPCA

Сделал еще один ран с такими логами [learn2.log](./learn2.log), результаты в среднем не выше. Видно, что алгоритмы переобучаются на cэмплированных данных, сэмплировать меньше не помогает. В репе старая версия параметров [prepare_dataset.py](./prepare_dataset.py), раз новые не дали результатов лучше


Выбрал 4 лучших классификатора с лучшими параметрами из первого рана, написал скрипт (ниже) для их обучения


\* - три цифры в конце - ROC на 5 фолдах при обучении, accuracy (для души), ROC на тестовой выборке

In [3]:
from imblearn.over_sampling import ADASYN
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("./dataset.csv")

X = df.drop(["target"], axis=1)
y = df["target"]

scaler = StandardScaler()

X = scaler.fit_transform(X, y)

# split first, then resample
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RS
)

over_sampler = ADASYN(random_state=RS)
reductor = KernelPCA(kernel="poly", n_components=5, random_state=RS)

In [4]:
from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    RandomForestClassifier,
)
from sklearn.neighbors import KNeighborsClassifier

bagging_clf = BaggingClassifier(
    estimator=RandomForestClassifier(random_state=RS), n_estimators=50, random_state=RS
)
rf_clf = RandomForestClassifier(criterion="entropy", max_depth=None, random_state=RS)
knn_clf = KNeighborsClassifier(n_neighbors=5)
adaboost_clf = AdaBoostClassifier(
    estimator=RandomForestClassifier(), n_estimators=25, random_state=RS
)

In [5]:
from sklearn.metrics import accuracy_score, make_scorer, roc_auc_score
from sklearn.model_selection import cross_val_score

X_train_resampled, y_train_resampled = over_sampler.fit_resample(X_train, y_train)

X_reducted = reductor.fit_transform(X_train_resampled)

for est in [bagging_clf, rf_clf, knn_clf, adaboost_clf]:
    print(est)
    est.fit(X_reducted, y_train_resampled)

    print(
        cross_val_score(
            est,
            X_reducted,
            y_train_resampled,
            scoring=make_scorer(roc_auc_score, average="weighted"),
            cv=5,
        )
    )

    X_pred = reductor.transform(X_test)
    y_pred = est.predict(X_pred)

    print(roc_auc_score(y_true=y_test, y_score=y_pred, average="weighted"))

BaggingClassifier(estimator=RandomForestClassifier(random_state=42),
                  n_estimators=50, random_state=42)
[0.83087921 0.8030303  0.7854696  0.8328858  0.80889389]
0.6988304093567251
RandomForestClassifier(criterion='entropy', random_state=42)
[0.83792147 0.81244244 0.78662084 0.83525855 0.82309035]
0.6989696463380675
KNeighborsClassifier()
[0.79092818 0.74021722 0.74768628 0.78555747 0.77697157]
0.7039821776663883
AdaBoostClassifier(estimator=RandomForestClassifier(), n_estimators=25,
                   random_state=42)
[0.84146506 0.81609834 0.79257924 0.84709697 0.81834506]
0.6979601782233361


В итоге лучше всего работают 3 ансамблевых классификатора и KNN:

|   | classifier                                                              | AUC score |
|---|-------------------------------------------------------------------------|-----------|
| 1 | KNeighborsClassifier(n_neighbours=5)                                    | 0.70398   |
| 2 | RandomForestClassifier(criterion='entropy')                             | 0.69897   |
| 3 | BaggingClassifier(estimator=RandomForestClassifier(),n_estimators=50)   | 0.69883   |
| 4 | AdaBoostClassifier(estimator=RandomForestClassifier(), n_estimators=25) | 0.69796   |


In [6]:
test_df = pd.read_csv("test.csv", index_col="id")
test_df

,0,1,2,3,4,5,6,7,8,9,...,795,796,797,798,799,800,801,802,803,804
id,,,,,,,,,,,,,,,,,,,,,
0,5.351105,1.933021,2.947073,-1209.792924,-1590.254772,1.963682,0.112038,775.516167,0.424224,-0.028495,...,905.747225,4.499702,0.803534,15.112012,1.902430,66.524220,1.107305,1.316054,-1343.460664,97.308018
1,567.987628,3.806155,569.333678,-1420.337690,-341.343355,1.996925,-0.110698,4.487823,562.999437,0.981713,...,159.436985,562.891324,-0.241692,15.766900,-0.058638,67.255965,562.758557,569.288638,236.871615,97.996751
2,-159.797745,0.004768,-152.936798,-1549.816964,1880.071160,-0.012737,1.070118,6.080241,-160.412191,0.124058,...,435.572638,-150.617012,0.003843,15.072682,-0.060233,66.337922,-158.522701,-160.181844,-46.134951,97.206372
3,7.477377,4.151975,2.659569,221.135099,1960.404537,0.978165,2.083902,5.732444,2.915118,-0.145356,...,1063.612083,7.678978,1.009009,15.884924,3.025117,67.526615,9.057854,8.072284,-506.328267,98.280454
4,0.556083,1.132735,8.633191,82.622337,-1694.039840,0.087522,1.971156,5.882287,6.228731,2.011331,...,-421.537996,8.360745,0.079631,15.893206,0.972241,67.213155,2.137631,4.197359,-188.658023,98.025160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,-541.034705,-0.008312,-544.615414,-1592.396354,-614.480167,1.134986,1.998570,7.803693,-539.858573,1.106142,...,-414.226747,-548.470147,1.244007,15.878753,-0.298417,67.191183,-546.863541,-545.404141,-216.376545,98.036042
796,3.753159,2.008486,1.454154,1089.738760,-357.088414,2.021897,0.090920,2.439324,2.813008,-0.018248,...,1226.115548,6.687068,0.169984,15.806151,4.122578,67.021953,4.376122,1.985985,-1434.617024,97.796732
797,9.113064,-0.216186,4.032969,446.758874,564.753214,-0.094708,2.029986,888.569158,0.557762,1.036690,...,320.664389,1.473778,1.839615,14.320555,1.124796,65.602284,2.076574,9.043637,757.438908,96.498003


In [7]:
final_X_test_scaled = scaler.transform(test_df)
final_X_test_reducted = reductor.transform(final_X_test_scaled)
final_y_test_pred = knn_clf.predict(final_X_test_reducted)

In [8]:
submission = pd.DataFrame(final_y_test_pred, columns=["target"])
submission.to_csv("submission.csv", index_label="id")

In [9]:
print(submission.value_counts())
submission

target
0         573
1         227
Name: count, dtype: int64


,target
0,0
1,1
2,0
3,1
4,1
...,...
795,0
796,0
797,0
798,0
